In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
base_path = '../../data/split/with-outliers/combined'

static_train = pd.read_csv(f'{base_path}/static_train.csv')
static_val = pd.read_csv(f'{base_path}/static_val.csv')

static_train, static_test = train_test_split(static_train, test_size=0.1, random_state=0, stratify=static_train['los_icu_binned'])

In [3]:
static_binned = pd.read_csv('../../data/static_cleaned.csv')

static_binned.head()

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,los_icu_binned
0,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,...,0,1,1,0,0,1,1,0,0,2 to 4 days
1,29179088,4/10/80 23:21,4/17/80 14:00,4/12/80 17:41,4/16/80 13:38,3.83,0,1,WHITE,72.276455,...,1,1,1,0,1,0,1,1,0,2 to 4 days
2,27139242,4/25/33 5:47,5/13/33 7:15,4/25/33 6:37,4/27/33 16:01,2.39,0,1,BLACK/AFRICAN AMERICAN,73.312782,...,1,0,0,0,0,0,0,0,0,2 to 4 days
3,25918892,10/27/43 15:06,10/31/43 14:21,10/27/43 15:06,10/30/43 12:11,2.88,0,1,WHITE,91.820358,...,0,0,0,0,0,0,1,1,1,2 to 4 days
4,20778724,10/4/67 7:15,10/19/67 17:45,10/5/67 9:23,10/6/67 21:43,1.51,0,1,ASIAN - CHINESE,63.756003,...,0,1,1,0,0,0,1,0,1,1 to 2 days


In [4]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = ohe.fit_transform(static_binned[['los_icu_binned']])
ohe_cols = ohe.get_feature_names_out(['los_icu_binned'])

ohe_los = pd.DataFrame(encoded, columns=ohe_cols)
static_ohe = pd.concat([static_binned, ohe_los], axis=1)

static_ohe.drop('los_icu_binned', axis=1, inplace=True)

static_ohe.head()

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,ckd,cld,copd,diabetes,hypertension,ihd,stroke,los_icu_binned_1 to 2 days,los_icu_binned_2 to 4 days,los_icu_binned_4+ days
0,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,...,1,0,0,1,1,0,0,0.0,1.0,0.0
1,29179088,4/10/80 23:21,4/17/80 14:00,4/12/80 17:41,4/16/80 13:38,3.83,0,1,WHITE,72.276455,...,1,0,1,0,1,1,0,0.0,1.0,0.0
2,27139242,4/25/33 5:47,5/13/33 7:15,4/25/33 6:37,4/27/33 16:01,2.39,0,1,BLACK/AFRICAN AMERICAN,73.312782,...,0,0,0,0,0,0,0,0.0,1.0,0.0
3,25918892,10/27/43 15:06,10/31/43 14:21,10/27/43 15:06,10/30/43 12:11,2.88,0,1,WHITE,91.820358,...,0,0,0,0,1,1,1,0.0,1.0,0.0
4,20778724,10/4/67 7:15,10/19/67 17:45,10/5/67 9:23,10/6/67 21:43,1.51,0,1,ASIAN - CHINESE,63.756003,...,1,0,0,0,1,0,1,1.0,0.0,0.0


In [5]:
static_train_ohe = static_ohe[static_ohe['id'].isin(static_train['id'])]
static_val_ohe = static_ohe[static_ohe['id'].isin(static_val['id'])]
static_test_ohe = static_ohe[static_ohe['id'].isin(static_test['id'])]

In [6]:
notes = pd.read_csv('../../data/notes_cleaned.csv')
notes_train = notes[notes['id'].isin(static_train['id'])]
notes_val = notes[notes['id'].isin(static_val['id'])]
notes_test = notes[notes['id'].isin(static_test['id'])]

In [7]:
dynamic = pd.read_csv('../../data/dynamic_cleaned.csv')
dynamic_train = dynamic[dynamic['id'].isin(static_train['id'])].copy()
dynamic_val = dynamic[dynamic['id'].isin(static_val['id'])].copy()
dynamic_test = dynamic[dynamic['id'].isin(static_test['id'])].copy()

In [8]:
static_train_ohe.id.nunique()

6980

In [9]:
dynamic_train.id.nunique()

6980

In [10]:
notes_train.id.nunique()

6980

In [11]:
static_val_ohe.id.nunique()

1940

In [12]:
dynamic_val.id.nunique()

1940

In [13]:
notes_val.id.nunique()

1940

In [14]:
static_test_ohe.id.nunique()

776

In [15]:
dynamic_test.id.nunique()

776

In [16]:
notes_test.id.nunique()

776

In [17]:
all(static_val_ohe.id.isin(dynamic_val.id))

True

In [18]:
base_path = '../../data/split/with-outliers/combined/one-hot-encoded'

static_train_ohe.to_csv(f'{base_path}/static_train.csv', index=False)
static_val_ohe.to_csv(f'{base_path}/static_val.csv', index=False)
static_test_ohe.to_csv(f'{base_path}/static_test.csv', index=False)